In [8]:
import breakwater as bw
import pandas as pd
import os
from pathlib import Path
import numpy as np

In [9]:
from development_overtopping_DKA import (eurotop2018_6_5, 
                                         surf_similarity, 
                                         gamma_beta_eurotop_2018_6_9,
                                         eurotop_6_5_q, 
                                         calc_beta)
from development_armour_stability_DKA import (vangent_armour_reduction, 
                                              hudson_fixed_slope, 
                                              rock_manual_5_196, 
                                              rock_manual_5_195, 
                                              rock_manual_5_194,
                                              rock_manual_5_194_Sd,
                                              vandermeer_shallow_Sd)
from development_material_DKA import(get_class,
                                     get_Dn50)

from development_scour_DKA import(sumer_fredsoe,
                                 v_scour)
                                 
from development_toe_DKA import (toe_stability_Nod)

from breakwater.utils.exceptions import user_warning

In [10]:
project_data = pd.read_excel(Path("./Input data/") / "test_data_phase_II.xlsx",
                             index_col = 1,
                             sheet_name='Input_Project specific')
requirements_data = pd.read_excel(Path("./Input data/") / "test_data_phase_II.xlsx",
                                  index_col = 0,
                                  sheet_name='Input_requirements')
wave_data = pd.read_excel(Path("./Input data/") / "test_data_phase_II.xlsx",
                          index_col = 0,
                          sheet_name='input_hydrotechnical',
                         skiprows = 1)
cross_section_data = pd.read_excel(Path("./Input data/") / "test_data_phase_II.xlsx", 
                                   sheet_name='Input_Cross section',
                                  skiprows = 1)
concrete_element_data = pd.read_excel(Path("./Input data/") / "test_data_phase_II.xlsx", 
                                      sheet_name='Input_concrete_elements',
                                      index_col = 0,
                                      skiprows = 1)
gradings_data = pd.read_excel(Path("./Input data/") / "test_data_phase_II.xlsx", 
                                      sheet_name='input_rock_gradings',
                                      index_col = 0,
                                      skiprows = 2)
cross_section_validation_data = pd.read_excel(Path("./Input data/") / "test_data_phase_II.xlsx", 
                                              sheet_name='input_cross_section_validation',
                                              skiprows = 1)

c:\ProgramData\Anaconda3\envs\env_bw\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\ProgramData\Anaconda3\envs\env_bw\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [4]:
#project_data

In [5]:
#requirements_data

In [6]:
#wave_data

In [7]:
#cross_section_data

In [8]:
#gradings_data

In [11]:
# VALIDATE OVERTOPPING REQUIREMENT

# Do we want to put this entire part in a function to keep the scripts clean a bit?


Calculation_case = 59
Cross_section_id = 0
access_state = 'restricted'

# Open project specific parameters
g = project_data.at['g', 'Value']

# Open structure specific parameters
tana          = cross_section_data.at[Cross_section_id, 'tan_a']
dir_structure = cross_section_data.at[Cross_section_id, 'dir_structure']
safety        = cross_section_data.at[Cross_section_id, 'safety']

# Get info for sea state. Currently implemented to do only the first sea state
Hm0      = wave_data.at[Calculation_case, 'Hm0']
wl       = wave_data.at[Calculation_case, 'wl']
Tm_min_1 = wave_data.at[Calculation_case, 'Tm-1,0']
dir_wave = wave_data.at[Calculation_case, 'dir_wave']

# Get allowed q for the location and calculation case
LS        = wave_data.at[Calculation_case, 'Limit State']
q_allowed = requirements_data.at[f'Overtopping limit {access_state} access', LS]

# Get data to validate
z_crest       = cross_section_validation_data.at[Cross_section_id, 'z_crest']
Rc = z_crest-wl

#Calculate roughness reduction
# Set armour for rougness reduction
armour_layer = 'Rock'
layers       = 2
permeability = 'permeable'

# Calculate surf-similarity with Hm0 and Tm-1,0
xi_m_min_1 = surf_similarity(tana, Hm0, Tm_min_1, g)

gamma_f = bw.core.overtopping.gamma_f(armour_layer, 
                                      xi_m_min_1, 
                                      layers = layers, 
                                      permeability = permeability) 

#Calculate obliqueness reduction. Function based on SAWP-#3504459-V48-IHS-COA-xxx-CAL_Armour_Stability_under_Waves.XLSM
beta = calc_beta(dir_structure, dir_wave)
gamma_beta = gamma_beta_eurotop_2018_6_9(beta)


q = eurotop_6_5_q(g, Hm0, Rc, gamma_f, gamma_beta, limit = False, safety=safety)

In [29]:
# VALIDATE CHOSEN ARMOUR CLASS (ROCK ARMOUR)
# To-do:
# - Hs/Hm0 determination

Calculation_case = 66
Cross_section_id = 1
# Armour = 2 layers rock, permeable core

# Open project specific parameters
g              = project_data.at['g'             , 'Value']
rho_a          = project_data.at['rho_r'         , 'Value']
rho_w          = project_data.at['rho_w'         , 'Value']
Cpl_shallow    = project_data.at['Cpl_shallow'   , 'Value']
Cs_shallow     = project_data.at['Cs_shallow'    , 'Value']
c_beta         = project_data.at['c_beta'        , 'Value']
c_rh           = project_data.at['c_rh'          , 'Value']
beta_max       = project_data.at['beta_max'      , 'Value']
Storm_duration = project_data.at['storm_duration', 'Value']
Safety         = project_data.at['sf_vdm'        , 'Value'] #Safety factor on top of regular VDM constants

#Get info for sea state
Hm0      = wave_data.at[Calculation_case, 'Hm0']
wl       = wave_data.at[Calculation_case, 'wl']
Tm_min_1 = wave_data.at[Calculation_case, 'Tm-1,0']
dir_wave = wave_data.at[Calculation_case, 'dir_wave']
Tm_0_2   = wave_data.at[Calculation_case, 'Tm0,2']
os_bin   = wave_data.at[Calculation_case, 'Offshore bin']
N        = np.round(Storm_duration*3600/Tm_0_2)


# Open structure specific parameters
tana            = cross_section_data.at[Cross_section_id, 'tan_a']
dir_structure   = cross_section_data.at[Cross_section_id, 'dir_structure']
z_bed           = cross_section_data.at[Cross_section_id, 'z_bed']
slope_foreshore = cross_section_data.at[Cross_section_id, 'slope_foreshore']
P               = cross_section_data.at[Cross_section_id, 'P']
rh              = cross_section_data.at[Cross_section_id, 'roundhead']

# Get allowed Sd for the location and calculation case
LS         = wave_data.at[Calculation_case, 'Limit State']
Sd_allowed = requirements_data.at['Armour damage limit', LS]

# Get data to validate
# Quit if the armour type is something else than 'Rock', because then this cell cannot validate the Dn50
armour_type = cross_section_validation_data.at[Cross_section_id, 'Armour_type']

if  armour_type == 'Rock':
    armour_class   = cross_section_validation_data.at[Cross_section_id, 'Armour_class']
    Dn50 = get_Dn50(armour_class, gradings_data, 'Min')
else:
    user_warning(f'Armour for this cross section {armour_type} and cannot be validated as Rock')
    Dn50 = 0.001


#Intermediate calculations
h              = wl-z_bed
Delta          = (rho_a-rho_w)/rho_w
alpha          = np.arctan(tana)
waveinfo       = bw.BattjesGroenendijk(Hm0, h, slope_foreshore)
H2_per         = waveinfo.get_Hp(0.02)
Hs             = waveinfo.get_Hn(3) #NU BATTJES-GROENENDIJK VOOR Hs UIT Hmo. IS DAT WAT WE WILLEN?

#OVERRIDE wave characteristics BECAUSE W+B sheet tales Hs = Hm0
Hs = Hm0
xi_s_0_2     = surf_similarity(tana, Hs, Tm_0_2, g)

# Check validity of Van der Meer shallow
vdm_shallow_validity = h/Hs

#Calculate obliqueness reduction. Function based on SAWP-#3504459-V48-IHS-COA-xxx-CAL_Armour_Stability_under_Waves.XLSM
beta = calc_beta(dir_structure, dir_wave)
gamma_beta  = vangent_armour_reduction(beta, c_beta, beta_max)

#Select correction factor (gamma) to apply according to DAR
if rh == "Yes":
    gamma_selected = c_rh
elif rh == "No":
    gamma_selected = gamma_beta
else:
    user_warning(f"Incorrect input in Roundhead indication")
    gamma_selected = 10000



#Calculate damage number
Sd = vandermeer_shallow_Sd(Hs, 
                            H2_per, 
                            Delta, 
                            P, 
                            Dn50, 
                            N, 
                            xi_s_0_2, 
                            alpha,
                            gamma_selected, 
                            Cpl = Cpl_shallow, 
                            Cs = Cs_shallow, 
                            safety = Safety)



In [15]:
#VALIDATE REAR SIDE ARMOUR STABILITY
# This calculation needs a crest height as input and therefore needs to be done after calculation of 
# the required crest height. If the calculation is done before choosing the final crest height it can give conservative
# results since the crest height in the calculation is lower than the final crest height.
# The calculation can also be performed in the verification stage only.

#To-do
# Hs-Hm0 interpretation

from numpy import cross


Calculation_case = 66
Cross_section_id = 1
# Armour = 2 layers rock, permeable core

# Open project specific parameters
g               = project_data.at['g'              , 'Value']
rho_a           = project_data.at['rho_r'          , 'Value']
rho_w           = project_data.at['rho_w'          , 'Value']
slope_roughness = project_data.at['slope_roughness', 'Value']
Storm_duration  = project_data.at['storm_duration' , 'Value']

#Get info for sea state
Hm0      = wave_data.at[Calculation_case, 'Hm0']
dir_wave = wave_data.at[Calculation_case, 'dir_wave']
Tm_0_2   = wave_data.at[Calculation_case, 'Tm0,2']
Tm_min_1 = wave_data.at[Calculation_case, 'Tm-1,0']
wl       = wave_data.at[Calculation_case, 'wl']
N        = np.round(Storm_duration*3600/Tm_0_2)

# Open structure specific parameters
tana            = cross_section_data.at[Cross_section_id, 'tan_a']
dir_structure   = cross_section_data.at[Cross_section_id, 'dir_structure']
cota_rear       = cross_section_data.at[Cross_section_id, 'cot_a_rear']
crest_roughness = cross_section_data.at[Cross_section_id, 'crest_roughness']
B               = cross_section_data.at[Cross_section_id, 'B']
z_bed           = cross_section_data.at[Cross_section_id, 'z_bed']
slope_foreshore = cross_section_data.at[Cross_section_id, 'slope_foreshore']

# Get validation data
z_crest         = cross_section_validation_data.at[Cross_section_id, 'z_crest']
grading_rear    = cross_section_validation_data.at[Cross_section_id, 'rear_armour_class']


# Get allowed Sd for the location and calculation case
LS         = wave_data.at[Calculation_case, 'Limit State']
Sd_allowed = requirements_data.at['Armour damage limit', LS]

#Intermediate calculations
h              = wl-z_bed
Rc             = z_crest - wl
beta = calc_beta(dir_structure, dir_wave)
Dn50_rear      = get_Dn50(grading_rear, gradings_data, 'Min')

waveinfo       = bw.BattjesGroenendijk(Hm0, h, slope_foreshore)
H2_per         = waveinfo.get_Hp(0.02)
Hs             = waveinfo.get_Hn(3) #NU BATTJES-GROENENDIJK VOOR Hs UIT Hmo. IS DAT WAT WE WILLEN?

#OVERRIDE wave characteristics BECAUSE W+B sheet tales Hs = Hm0
Hs = Hm0 #TO BE ADAPTED IF WE CHOOSE TO DIFFERENTIATE BETWEEN Hs AND Hm0

Delta          = (rho_a-rho_w)/rho_w
xi_s_min_1     = surf_similarity(tana, Hs, Tm_min_1, g)

# Actual calculations
Ru_1_percent, gamma_f, gamma_beta, gamma  = rock_manual_5_196(Hs, xi_s_min_1, slope_roughness, beta)
u_1_percent = rock_manual_5_195(g, slope_roughness, crest_roughness, Ru_1_percent, Rc, B, Hs)


Sd_rear = rock_manual_5_194_Sd(Dn50_rear, N, u_1_percent, Tm_min_1, Delta, cota_rear, Rc, Hs)



In [9]:
# VALIDATE TOE STABILITY

Calculation_case = 60
Cross_section_id = 0

# Open project specific parameters
g            = project_data.at['g'           , 'Value']
rho_a        = project_data.at['rho_r'       , 'Value']
rho_w        = project_data.at['rho_w'       , 'Value']
Lt           = project_data.at['Lt'          , 'Value']
t_underlayer = project_data.at['t_underlayer', 'Value']

#Get info for sea state
Hm0      = wave_data.at[Calculation_case, 'Hm0']
wl       = wave_data.at[Calculation_case, 'wl']

# Open structure specific parameters
z_bed           = cross_section_data.at[Cross_section_id, 'z_bed']

# Get allowed Sd for the location and calculation case
LS          = wave_data.at[Calculation_case, 'Limit State']
Nod_allowed = requirements_data.at['Toe damage limit', LS]

# Get properties to validate
grading_toe = cross_section_validation_data.at[Cross_section_id,'toe_armour_class']

# Intermediate parameters
h               = wl-z_bed
Delta          = (rho_a-rho_w)/rho_w
Dn50_toe_av = get_Dn50(grading_toe, gradings_data, 'Av')
Dn50_toe_min = get_Dn50(grading_toe, gradings_data, 'Min')

# Check if filter layer needs to be applied
if gradings_data.at[grading_toe, 'Toe underlayer'] in gradings_data.index:
    class_filter = gradings_data.at[grading_toe, 'Toe underlayer']
    Dn50_filter = get_Dn50(class_filter, gradings_data, 'Av')
    t_filter = Dn50_filter*Lt
else:
    t_filter = 0

toe_layer_thickness = t_underlayer + t_filter + Dn50_toe_av*Lt
ht = h - toe_layer_thickness

#TEMPORARY FIX TO BE IN LINE WITH PHASE II
Hs = Hm0

Nod = toe_stability_Nod(Hs, h, ht, Delta, Dn50_toe_min)

In [34]:
%reset